In [1]:
import json
import numpy as np
import math
from functools import partial
from scipy.optimize import minimize


PI = math.pi

with open("marker_pos_for_calibration.json", "r") as st_json:
	st_python = json.load(st_json)

In [2]:
st_python.keys()

dict_keys(['depth_camera_pos_list', 'mocap_pos_list'])

In [3]:
number_of_points = len(st_python['depth_camera_pos_list'])


number_of_points

36

In [4]:
depth_camera_pos_list = np.array(st_python['depth_camera_pos_list'])
mocap_pos_list = np.array(st_python['mocap_pos_list'])

In [5]:
mocap_pos_list,depth_camera_pos_list

(array([[-1.52858019e-01,  3.25751305e-03,  5.68240643e-01],
        [-1.14236951e-01,  4.44602966e-03,  5.65053463e-01],
        [-7.97431469e-02,  5.34105301e-03,  5.62750816e-01],
        [-5.05696535e-02,  5.93733788e-03,  5.62969208e-01],
        [-1.83968246e-02,  6.47902489e-03,  5.60513020e-01],
        [ 1.33260489e-02,  6.92868233e-03,  5.57759047e-01],
        [ 4.42222953e-02,  7.55786896e-03,  5.57778597e-01],
        [ 9.05518234e-02,  8.57019424e-03,  5.49674988e-01],
        [ 1.29809484e-01,  9.20128822e-03,  5.45294762e-01],
        [ 1.34309947e-01,  9.37795639e-03,  5.07029533e-01],
        [ 1.06570445e-01,  9.03916359e-03,  5.02062798e-01],
        [ 6.85784966e-02,  8.34035873e-03,  4.98417377e-01],
        [ 3.88333499e-02,  7.87472725e-03,  4.90980148e-01],
        [ 5.06758690e-04,  7.08317757e-03,  4.89905834e-01],
        [-4.37125266e-02,  6.08539581e-03,  4.91316557e-01],
        [-8.56860876e-02,  5.61141968e-03,  4.92130518e-01],
        [-1.37805045e-01

In [280]:
np.transpose(depth_camera_pos_list)


array([[ 0.11177864,  0.07880703,  0.0374731 ,  0.0013968 , -0.03001965,
        -0.06338304, -0.0977354 , -0.1289141 , -0.17630479, -0.21652754,
        -0.21034667, -0.18024983, -0.1418272 , -0.11100712, -0.0718085 ,
        -0.02774195,  0.01476273,  0.06822482,  0.1034952 ,  0.14647432,
         0.14935565,  0.14686888,  0.10718115,  0.06475125,  0.01544523,
        -0.03541939, -0.0807856 , -0.12920986, -0.17506284, -0.16697778,
        -0.13170764, -0.09285755, -0.05911789, -0.02867437,  0.00929568,
         0.07729195],
       [-0.10156959, -0.10510632, -0.09946088, -0.09576339, -0.09382102,
        -0.08863553, -0.08396334, -0.08201165, -0.07274118, -0.06623147,
        -0.03686669, -0.03541851, -0.03566345, -0.03183359, -0.03311993,
        -0.03701861, -0.04136599, -0.04547895, -0.04572118, -0.03321129,
         0.00699082,  0.04235803,  0.04618359,  0.04551494,  0.04957487,
         0.05498705,  0.06052729,  0.06685486,  0.06616684,  0.08865096,
         0.09191661,  0.09326

In [281]:
depth_camera_transformation = np.zeros((4,number_of_points))
mocap_transformation = np.zeros((4,number_of_points))

depth_camera_transformation[:3,:] = np.transpose(depth_camera_pos_list)
depth_camera_transformation[3,:]  = np.ones_like(depth_camera_transformation[3,:])

mocap_transformation[:3,:] = np.transpose(mocap_pos_list)
mocap_transformation[3,:]  = np.ones_like(mocap_transformation[3,:])

depth_camera_transformation

array([[ 0.11177864,  0.07880703,  0.0374731 ,  0.0013968 , -0.03001965,
        -0.06338304, -0.0977354 , -0.1289141 , -0.17630479, -0.21652754,
        -0.21034667, -0.18024983, -0.1418272 , -0.11100712, -0.0718085 ,
        -0.02774195,  0.01476273,  0.06822482,  0.1034952 ,  0.14647432,
         0.14935565,  0.14686888,  0.10718115,  0.06475125,  0.01544523,
        -0.03541939, -0.0807856 , -0.12920986, -0.17506284, -0.16697778,
        -0.13170764, -0.09285755, -0.05911789, -0.02867437,  0.00929568,
         0.07729195],
       [-0.10156959, -0.10510632, -0.09946088, -0.09576339, -0.09382102,
        -0.08863553, -0.08396334, -0.08201165, -0.07274118, -0.06623147,
        -0.03686669, -0.03541851, -0.03566345, -0.03183359, -0.03311993,
        -0.03701861, -0.04136599, -0.04547895, -0.04572118, -0.03321129,
         0.00699082,  0.04235803,  0.04618359,  0.04551494,  0.04957487,
         0.05498705,  0.06052729,  0.06685486,  0.06616684,  0.08865096,
         0.09191661,  0.09326

In [282]:

def t2pr(T=np.zeros((4,4))):
    p = T[0:3,3]
    R = T[0:3,0:3]
    return (p,R)

def t2r(T=np.zeros((4,4))):
    R = T[0:3,0:3]
    return R

def t2p(T=np.zeros((4,4))):
    p = T[0:3,3]
    return p

def pr2t(p=np.zeros((1,3)), R=np.zeros((4,4))):
    T = np.zeros((4,4))
    T[0:3,0:3] = R[:]
    T[0:3,3] =  p[:]
    T[3,3] = 1
    return T

def rpy2r(rpys=np.zeros(3)): # [radian]
    R = np.zeros((3,3))
    r = rpys[0]
    p = rpys[1]
    y = rpys[2]
    R[0,:] = np.vstack([
        np.cos(y)*np.cos(p),
        -np.sin(y)*np.cos(r) + np.cos(y)*np.sin(p)*np.sin(r),
        np.sin(y)*np.sin(r) + np.cos(y)*np.sin(p)*np.cos(r)
        ]).reshape(3)
    R[1,:] = np.vstack([
        np.sin(y)*np.cos(p),
        np.cos(y)*np.cos(r) + np.sin(y)*np.sin(p)*np.sin(r),
        -np.cos(y)*np.sin(r) + np.sin(y)*np.sin(p)*np.cos(r)
        ]).reshape(3)
    R[2,:] = np.vstack([
        -np.sin(p),
        np.cos(p)*np.sin(r),
        np.cos(p)*np.cos(r)
        ]).reshape(3)
    return R

def r2rpy(R): # [radian]
    r = np.arctan2(R[2,1],R[2,2])
    p = np.arctan2(-R[2,0],np.sqrt(R[2,1]*R[2,1]+R[2,2]*R[2,2]))
    y = np.arctan2(R[1,0],R[0,0])
    rpys = np.stack([r,p,y])
    return rpys

def skew(p=np.zeros(3)):
    skew_ps = np.zeros((3,3))
    
    skew_ps[0,:] = np.vstack([0, -p[:,2], p[:,1]]).transpose(0,1)
    skew_ps[1,:] = np.vstack([p[:,2], 0, -p[:,0]]).transpose(0,1)
    skew_ps[2,:] = np.vstack([-p[:,1], p[:,0],0]).transpose(0,1)
    return skew_ps



In [283]:
T_init = mocap_transformation@ np.transpose(depth_camera_transformation) @np.linalg.inv(depth_camera_transformation@np.transpose(depth_camera_transformation))
t2p(T_init),r2rpy(t2r(T_init))

(array([1.2363415 , 0.05350178, 0.91736669]),
 array([-1.83922854,  0.03237427, -3.12201292]))

In [284]:
def get_position_orientation_loss(rpy_pos):
    rpy = rpy_pos[:3]
    p = rpy_pos[3:]
    R = rpy2r(rpy)
    T = pr2t(p,R)
    
    T_predicted = T @ depth_camera_transformation
    error_pos = T_predicted - mocap_transformation
    loss = np.mean(np.abs(error_pos[:3]))
    return loss

def get_loss_Fn(kwarg):
    keys = kwarg.keys()
    assert len(keys)==1
    
    key = list(keys)[0]
    value = kwarg[key]
    
    if key=='rpy':
        def p_loss(p):
            rpy = value
            R = rpy2r(rpy)
            T = pr2t(p,R)
            
            T_predicted = T @ depth_camera_transformation
            error_pos = T_predicted - mocap_transformation
            loss = np.mean(np.abs(error_pos[:3]))
            return loss
        return p_loss
    if key=='p':
        def rpy_loss(rpy):
            p   = value
            R = rpy2r(rpy)
            T = pr2t(p,R)
            
            T_predicted = T @ depth_camera_transformation
            error_pos = T_predicted - mocap_transformation
            loss = np.mean(np.abs(error_pos[:3]))
            return loss
        return rpy_loss
    else: raise Exception("kwargs key must contain 'rpy' or 'p'")




In [285]:
rpy_init = r2rpy(t2r(T_init))
p_init = t2p(T_init)
rpy_pos_init = np.concatenate([rpy_init,p_init])
print(get_position_orientation_loss(rpy_pos_init))

p_loss = get_loss_Fn({'rpy':rpy_init})
print(p_loss(p_init))
rpy_loss = get_loss_Fn({'p':p_init})
print(rpy_loss(rpy_init))

0.7454802624691459
0.7454802624691459
0.7454802624691459


In [256]:
rpy = rpy_init
p   = p_init


for _ in range(100):
    # optimizie p
    p_loss = get_loss_Fn({'rpy':rpy})
    res = minimize(p_loss, p, method='nelder-mead',
                options={'fatol': 1e-17, 'disp': True, 'maxiter':10_000})
    p = res.x

    rpy_loss = get_loss_Fn({'p':p})
    # optimizie rpy
    res = minimize(rpy_loss, rpy, method='nelder-mead',
                options={'fatol': 1e-17, 'disp': True, 'maxiter':10_000})
    rpy = res.x


rpy_pos = np.concatenate([rpy, p])
get_position_orientation_loss(rpy_pos)

p = p
R = rpy2r(rpy)
T = pr2t(p,R)

T_predicted = T @ depth_camera_transformation
error_pos = T_predicted - mocap_transformation

# np.transpose(error_pos)[0]
np.max(error_pos)

0.15381993565762664

In [290]:
rpy_pos = rpy_pos_init
res = minimize(get_position_orientation_loss, rpy_pos, method='nelder-mead',
               options={'fatol': 1e-17, 'disp': True, 'maxiter':10_000})

rpy_pos = res.x
rpy = rpy_pos[:3]
p = rpy_pos[3:]
R = rpy2r(rpy)
T = pr2t(p,R)

T_predicted = T @ depth_camera_transformation
error_pos = T_predicted - mocap_transformation

# np.transpose(error_pos)[0]
np.max(error_pos)

Optimization terminated successfully.
         Current function value: 0.035046
         Iterations: 1005
         Function evaluations: 1629


0.05860692915714205